<a href="https://colab.research.google.com/github/MLandML/MLandML/blob/learning_projects/Stock_prices_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Input,LSTM,GRU,Dense,GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io

df = pd.read_csv(io.BytesIO(uploaded['SBUX.csv']))
df.head()

In [ ]:
df = df.drop("Adj Close",axis=1)

In [ ]:
df['Prev Close'] = df['Close'].shift(1)
df

In [ ]:
#return
df['Return'] = (df['Close'] - df['Prev Close'])/df['Prev Close']
df['Return'].hist()

In [ ]:
series = df['Return'].values[1:].reshape(-1,1)

scaler = StandardScaler()
scaler.fit(series[:len(series)//2])
series = scaler.transform(series).flatten()

In [ ]:
T = 10
D = 1
X = []
Y = []

for t in range(len(series)-T):
  x = series[t:t+T]
  y = series[t+T]
  X.append(x)
  Y.append(y)

X = np.array(X).reshape(-1,T,D)
Y = np.array(Y)
N = len(x)
X.shape

In [ ]:
i = Input(shape = (T,D))
x = LSTM(128)(i)
x = Dense(1)(x)

model = Model(i,x)
model.compile(
    loss = 'mse',
    optimizer = Adam(lr=0.01),
)

r = model.fit(X[:N//2],Y[:N//2],epochs=50,validation_data=(X[N//2:],Y[N//2:]))

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
#one-step forecasting
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:,0]

plt.plot(Y,label='Targets')
plt.plot(predictions,label='predictions')
plt.legend()

In [ ]:
#multi-step forecasting
val_targets = Y[N//2:]
val_preds = []
last_x = X[N//2]

while len(val_preds)<len(val_targets):
  p = model.predict(last_x.reshape(1,-1,1))[0,0]
  val_preds.append(p)
  last_x = np.roll(last_x,-1)
  last_x[-1] = p

plt.plot(val_targets,label='Targets')
plt.plot(val_preds,label='forecasted_predictions')
plt.legend()

In [ ]:
#using all the columns

input_data = df[['High','Low','Open','Close','Volume']].values
targets = df['Return'].values

In [ ]:
T = 10
D = input_data.shape[1]
N = len(input_data) - T

In [ ]:
NTrain = len(input_data) * 2//3
scaler = StandardScaler()
scaler.fit(input_data[:NTrain+T])
input_data = scaler.transform(input_data)

X_train = np.zeros((NTrain,T,D))
Y_train = np.zeros(NTrain)

X_test = np.zeros((N-NTrain,T,D))
Y_test = np.zeros(N-NTrain)

for t in range(NTrain):
  X_train[t,:,:] = input_data[t:t+T]
  Y_train[t] = (targets[t+T]>0)

for u in range(N-NTrain):
  t = u + NTrain
  X_test[u,:,:] = input_data[t:t+T]
  Y_test[u] = (targets[t+T]>0)

In [ ]:
i = Input(shape = (T,D))
x = LSTM(64)(i)
x = Dense(1,activation='sigmoid')(x)

model = Model(i,x)
model.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(lr=0.01),
    metrics =['accuracy']
)

r = model.fit(X_train,Y_train,epochs=100,validation_data=(X_test,Y_test))

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'],label='accuracy')
plt.plot(r.history['val_accuracy'],label='val_accuracy')
plt.legend()